In [227]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3


In-[period] and calendar grosses
Monthly, quarterly, season, and yearly charts can be viewed either by “in-[period] releases” or “calendar grosses”. Viewing by in-[period] releases shows total box office results for releases during that period, regardless of when the money was made. For example, viewing by in-year releases for 2019 shows box office results for all movies released in 2019. A movie released in 2018 will not appear on this chart even if it continued to earn money in 2019. Viewing by calendar grosses shows box office results within a given time period, regardless of when the movies were released. For example, viewing by calendar grosses for 2019 shows box office results for all movies that earned money in 2019. A movie released in 2018 may appear on this chart if it continued to earn money in 2019.

Calendar grosses are based on daily box office receipts or, when daily data is not available, estimates are used and are based on weekend and weekly data and historical box office trends. In cases where a final reported gross is different from its last reported gross for a given movie, the difference is assigned to the two weeks after the last reported gross date. In most cases, this reflects receipts that have trickled in after a movie has stopped being tracked for reportage. Since box office has been more closely tracked in recent years, the calendar gross data is generally considered more comprehensive after 2001, while pre-2001 estimates are considered approximate. Accuracy of calendar grosses improves over a wider range of time viewed.

In [211]:
for year in range(2012, 2023):
    globals()[f"box_office_{year}"] = pd.read_excel("IMDB_boxoffice_2022_2012.xlsx", sheet_name = str(year), na_values = "-" )

In [212]:
for year in range(2012, 2023):
    globals()[f"box_office_{year}"]["release_year"] = year

In [213]:
box_office_decade = pd.concat([box_office_2012,
           box_office_2013,
           box_office_2014,
           box_office_2015,
           box_office_2016,
           box_office_2017,
           box_office_2018,
           box_office_2019,
           box_office_2020,
           box_office_2021,
           box_office_2022], ignore_index=True)

In [214]:
box_office_decade.groupby("release_year").count()

,Rank,Release,Genre,Budget,Running Time,Gross,Max Th,Opening,% of Total,Open Th,Open,Close,Distributor,Estimated
release_year,,,,,,,,,,,,,,
2012,200,200,0,0,0,200,198,195,193,195,200,199,199,200
2013,200,200,0,0,0,200,200,199,198,199,200,198,197,200
2014,200,200,0,0,0,200,200,200,200,200,200,199,198,200
2015,200,200,0,0,0,200,200,198,194,198,200,199,198,200
2016,200,200,0,0,0,200,200,199,198,199,200,196,198,200
2017,200,200,0,0,0,200,200,196,196,196,200,198,198,200
2018,200,200,0,0,0,200,200,198,197,198,200,198,192,200
2019,200,200,0,0,0,200,199,197,196,197,200,136,196,200
2020,200,200,0,0,0,200,195,184,153,180,200,6,198,200


In [215]:
box_office_decade.rename(columns = {"Release": "primary_title"}, inplace= True)
box_office_decade.drop(columns = "Estimated", inplace = True)

In [216]:
con = sqlite3.connect('im.db')

cursor = con.cursor()


# Execute the query
# (This is a special query for finding the table names. You don't need to memorize it.)
cursor.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
# Fetch the result and store it in table_names

table_names = cursor.fetchall()

table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [217]:
movie_basics = pd.read_sql("""

SELECT *
FROM movie_basics


""", con)

In [218]:
movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [219]:
movies = pd.merge(box_office_decade, movie_basics, how = "inner", on = "primary_title")

In [220]:
movies.drop(columns= ["start_year", "original_title", "Genre", "Budget", "Running Time"], inplace= True)

In [224]:
movies["genre1"] = movies["genres"].str.split(',', expand = True)[0]
movies["genre2"] = movies["genres"].str.split(',', expand = True)[1]
movies["genre3"] = movies["genres"].str.split(',', expand = True)[2]
movies.drop(columns= ["genres"], inplace= True)

In [225]:
movies.sample(10)

,Rank,primary_title,Gross,Max Th,Opening,% of Total,Open Th,Open,Close,Distributor,release_year,movie_id,runtime_minutes,genre1,genre2,genre3
1204,115,Norm of the North,17062499,2411,"$6,844,137",40.1%,"2,411",2022-01-15,Mar 17,Lionsgate,2016,tt1594972,90.0,Adventure,Animation,Comedy
101,91,Frankenweenie,35291068,"3,005","$11,412,213",32.3%,"3,005",2022-10-05,Feb 28,Walt Disney Studios Motion Pictures,2012,tt1142977,87.0,Animation,Comedy,Family
349,90,The Mortal Instruments: City of Bones,31165421,3118,"$9,336,957",30%,"3,118",2022-08-21,Oct 6,Screen Gems,2013,tt1538403,130.0,Action,Fantasy,Horror
2434,179,Poser,42179,9,"$15,250",NaN,1,2022-06-03,NaN,Oscilloscope,2022,tt9000084,88.0,Crime,Drama,None
1535,152,The Book of Henry,4504974,650,"$1,424,540",31.6%,579,2022-06-16,Aug 17,Focus Features,2017,tt4572792,105.0,Crime,Drama,Thriller
1929,62,Rambo: Last Blood,44819352,"3,618","$18,872,919",42.1%,"3,618",2022-09-20,Nov 21,Lionsgate,2019,tt1206885,NaN,Action,Adventure,Thriller
1191,108,Middle School: The Worst Years of My Life,20007149,2822,"$6,878,437",34.4%,"2,822",2022-10-07,Dec 1,Lionsgate,2016,tt4981636,92.0,Comedy,Family,None
562,62,Muppets Most Wanted,51183113,3194,17005126,0.332,3194,2022-03-21,Jul 17,Walt Disney Studios Motion Pictures,2014,tt2281587,107.0,Adventure,Comedy,Crime
418,144,The Spectacular Now,6854611,770,"$197,415",2.9%,4,2022-08-02,Dec 12,A24,2013,tt1714206,95.0,Comedy,Drama,Romance
482,1,American Sniper,350126372,3885,633456,0.002,4,2022-12-25,Jun 25,Warner Bros.,2014,tt2179136,133.0,Action,Biography,Drama
